In [134]:
import numpy as np
import pandas as pd
import csv
import copy
import itertools

import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# Check the stats of all connections
def CheckTheEdgeStats(df):   
    print('# of edges: ', len(df))
    print('# of unique vertices: ', len(set(df.start).union(set(df.end))))
    print('# of time stamps: ', len(np.unique(df.timestamp)))
    print('# of unique start vertices: ', len(set(df.start)))
    print('# of unique end vertices: ', len(set(df.end)))

# Index of abnormal/normal connections 
def CheckTheEdgeLabels(df):   
    abnorm_idx = [idx for idx,val in enumerate(df.label) if val != '-']
    norm_idx = [idx for idx,val in enumerate(df.label) if val == '-']
    print('# of anomaly edges: ', len(abnorm_idx))
    print('# of normal edges: ', len(norm_idx))
    print('Percentage of anomaly edges: ', len(abnorm_idx) / (len(abnorm_idx) + len(norm_idx)))
    
    return abnorm_idx, norm_idx

In [3]:
# Load the MIDAS processed data
graphProc_df = pd.read_csv('../pyMIDAS-master/pyMIDAS-master/examples/darpa_midas.csv', header=None)
graphProc_df.columns = ['start', 'end', 'timestamp']

# Check the the data stats
CheckTheEdgeStats(graphProc_df)

# of edges:  4554344
# of unique vertices:  23398
# of time stamps:  960
# of unique start vertices:  9484
# of unique end vertices:  23398


In [38]:
### Load the original data and check the stats

# Load the original MIDAS data with labels
graphOrig_df = pd.read_csv('../pyMIDAS-master/pyMIDAS-master/examples/darpa_original.csv', header = None)
graphOrig_df.columns = ['start', 'end', 'timestamp', 'label']
CheckTheEdgeStats(graphOrig_df)
print('-----------------------------------------------------')

# Check the label stats
abnorm_idx, norm_idx = CheckTheEdgeLabels(graphOrig_df)
print('-----------------------------------------------------')

# Add the binary label to data
bi_labels = np.zeros(len(graphOrig_df))
bi_labels[abnorm_idx] = 1
graphOrig_df['label'] = bi_labels

# -----------------------------------------------------------------------------------------------
# Convert original IP and timestamps to indexes
all_ip = sorted(list(set(graphOrig_df.start).union(set(graphOrig_df.end))))
all_timestamp = sorted(list(set(graphOrig_df.timestamp)))

ipIdx_dict = {all_ip[idx]: idx for idx in range(len(all_ip))} 
timeIdx_dict = {all_timestamp[idx]: idx+1 for idx in range(len(all_timestamp))}

graphLab_df = copy.deepcopy(graphOrig_df)
graphLab_df['start'] = graphLab_df['start'].apply(lambda x: ipIdx_dict[x])
graphLab_df['end'] = graphLab_df['end'].apply(lambda x: ipIdx_dict[x])
graphLab_df['timestamp'] = graphLab_df['timestamp'].apply(lambda x: timeIdx_dict[x])
graphLab_df['label'] = bi_labels

# -----------------------------------------------------------------------------------------------
# Stats for repetitive connections at the same time unit
unique_conn_df = graphLab_df.groupby(['timestamp', 'start', 'end']).size().reset_index(name='Count')

print('# of Repetative Connections: ')
print('Max: ', max(unique_conn_df.Count))
print('Min: ', min(unique_conn_df.Count))
print('Mean: ', np.mean(unique_conn_df.Count))
print('Median: ', np.median(unique_conn_df.Count))
print('-----------------------------------------------------')

# Get the aggregated abnormal labels for connections 
label_num = graphLab_df.groupby(['timestamp', 'start', 'end']).apply(lambda x: sum(x['label'])).tolist()

# of edges:  4554344
# of unique vertices:  25525
# of time stamps:  46572
# of unique start vertices:  9484
# of unique end vertices:  23398
-----------------------------------------------------
# of anomaly edges:  2737209
# of normal edges:  1817135
Percentage of anomaly edges:  0.6010105955984001
# of Repetative Connections: 
Max:  10822
Min:  1
Mean:  8.720854173209048
Median:  1.0
-----------------------------------------------------


In [116]:
# All unique start ip
startID = sorted(np.unique(graphLab_df.start))
print('All starts: ', len(startID))

# Sample a subset of data for analysis
sample_num = 200
sample_flag = False

if sample_flag: 
    startID = startID[:sample_num]
print('Sampled starts: ', len(startID))

# ---------------------------------------------------------
# Generate the target ip, time difference, labels of trajectories 
traj_list, trajTimeDiff_list, trajLabs_list = [], [], [] 
for tmpID in startID: 
    tmp_traj = graphLab_df.loc[graphLab_df.start == tmpID]['end'].tolist()
    tmp_trajTimeDiff = [0] + np.diff(graphLab_df.loc[graphLab_df.start == tmpID]
                                      ['timestamp']).tolist()
    tmp_trajLabs = graphLab_df.loc[graphLab_df.start == tmpID]['label'].tolist()
    
    traj_list.append(tmp_traj)
    trajTimeDiff_list.append(tmp_trajTimeDiff) 
    trajLabs_list.append(tmp_trajLabs)
    
traj_len = [len(tmp_traj) for tmp_traj in traj_list]

All starts:  9484
Sampled starts:  9484


In [165]:
len_thres = 10

print('Check the length of trajectories: ')
fillter_idx = [idx for idx, val in enumerate(traj_len) if val >= len_thres]
print(len(fillter_idx) / len(traj_len))

filter_labs = [trajLabs_list[idx] for idx in fillter_idx]
filter_allLabs = list(itertools.chain.from_iterable(filter_labs))
print(filter_allLabs.count(1) / len(filter_allLabs))

Check the length of trajectories: 
0.1372838464782792
0.5982884140206707


In [171]:
graphLab_df.loc[(graphLab_df.start == 7577) & (graphLab_df.label == 1)]

start    end  timestamp  label
962634    7577  11443      19085    1.0
1006344   7577  11101      20215    1.0
1060641   7577  11101      21994    1.0
1951503   7577  11101      27635    1.0
3842993   7577  14218      40260    1.0
4460734   7577  11443      45387    1.0

In [177]:
graphLab_df.loc[graphLab_df.start == 11443]

start   end  timestamp  label
7        11443  8274          3    0.0
2899     11443  8117         71    0.0
3072     11443  7607         81    0.0
3073     11443  7918         81    0.0
3074     11443  8215         81    0.0
3075     11443  8274         81    0.0
3078     11443  7607         82    0.0
3079     11443  7751         82    0.0
3080     11443  7897         82    0.0
3081     11443  8215         82    0.0
3698     11443  8215         99    0.0
4175     11443  8215        105    0.0
5103     11443  8215        122    0.0
6006     11443  7918        133    0.0
6007     11443  8117        133    0.0
6591     11443  8017        139    0.0
6912     11443  7607        143    0.0
6913     11443  7751        143    0.0
6914     11443  7765        143    0.0
6915     11443  8215        143    0.0
6916     11443  8236        143    0.0
8046     11443  8215        160    0.0
8664     11443  7706        174    0.0
8665     11443  7863        174    0.0
8666     11443  7897        174    0.0
8667     11443  8236        174    0.0
8668     11443  8236        174    0.0
8792     11443  7607        177    0.0
8793     11443  7706        177    0.0
8794     11443  8235        177    0.0
...        ...   ...        ...    ...
4511258  11443  8235      45794    0.0
4512398  11443  8117      45808    0.0
4513240  11443  7897      45817    0.0
4513566  11443  8117      45821    0.0
4513567  11443  8117      45821    0.0
4513568  11443  8117      45821    0.0
4513569  11443  8117      45821    0.0
4513570  11443  8117      45821    0.0
4513571  11443  8117      45821    0.0
4513572  11443  8117      45821    0.0
4513747  11443  8117      45823    0.0
4513748  11443  8117      45823    0.0
4513749  11443  8117      45823    0.0
4513750  11443  8117      45823    0.0
4513751  11443  8117      45823    0.0
4513752  11443  8117      45823    0.0
4513753  11443  8117      45823    0.0
4514099  11443  8117      45826    0.0
4514100  11443  8117      45826    0.0
4514101  11443  8117      45826    0.0
4514102  11443  8117      45826    0.0
4514393  11443  8117      45830    0.0
4514394  11443  8117      45830    0.0
4514395  11443  8117      45830    0.0
4514396  11443  8117      45830    0.0
4514397  11443  8117      45830    0.0
4514870  11443  7706      45836    0.0
4519321  11443  8215      45904    0.0
4519322  11443  8235      45904    0.0
4521600  11443  7607      45945    1.0

[36200 rows x 4 columns]

In [178]:
graphLab_df.loc[graphLab_df.end == 11443]

start    end  timestamp  label
17        7706  11443          6    0.0
20        8017  11443          7    0.0
103       7765  11443         11    0.0
144       7897  11443         13    0.0
145       7897  11443         13    0.0
149       8235  11443         14    0.0
439       7706  11443         20    0.0
1611      7918  11443         38    0.0
1954      7918  11443         46    0.0
2520      8215  11443         60    0.0
2638      8017  11443         63    0.0
2639      8017  11443         63    0.0
2671      7765  11443         65    0.0
2882      7577  11443         71    0.0
2895      8117  11443         71    0.0
2896      8235  11443         71    0.0
2939      7607  11443         75    0.0
2946      7607  11443         76    0.0
2981      7863  11443         79    0.0
3010      8236  11443         79    0.0
3088      7918  11443         83    0.0
3505      8017  11443         97    0.0
3509      8235  11443         97    0.0
3582      7751  11443         98    0.0
3680      7577  11443         99    0.0
4199      7751  11443        107    0.0
4283      7607  11443        109    0.0
4338      7897  11443        110    0.0
4616      8274  11443        113    0.0
5091      7577  11443        122    0.0
...        ...    ...        ...    ...
4504252   7706  11443      45740    0.0
4504356   7577  11443      45741    0.0
4506976   8215  11443      45759    0.0
4507250   7765  11443      45761    0.0
4507254   7897  11443      45761    0.0
4507418   7577  11443      45762    0.0
4507557   8017  11443      45763    0.0
4507655   7765  11443      45764    0.0
4507757   8235  11443      45765    0.0
4508317   7706  11443      45770    0.0
4509415   7607  11443      45775    0.0
4509417   7765  11443      45775    0.0
4509512   7706  11443      45776    0.0
4509655   7765  11443      45778    0.0
4509742   7918  11443      45779    0.0
4510020   7577  11443      45782    0.0
4510534   7607  11443      45787    0.0
4511145   7897  11443      45793    0.0
4511597   7577  11443      45800    0.0
4512322   8117  11443      45808    0.0
4512879   8215  11443      45814    0.0
4513167   7577  11443      45817    0.0
4514563   7706  11443      45833    0.0
4514848   7577  11443      45836    0.0
4515189   8933  11443      45841    0.0
4519245   8556  11443      45903    0.0
4519294   7577  11443      45904    0.0
4521535   7577  11443      45945    0.0
4521679   7607  11443      45946    1.0
4521680   7607  11443      45946    1.0

[15729 rows x 4 columns]

In [180]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 11:30:02
